In [1]:
import random
known_weather_data = {
    'berlin': 20.0
}
def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]
    return round(random.uniform(-5, 35), 1)

In [2]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Get the current temperature for a specified city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city for which to retrieve the temperature"
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

In [3]:
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Set the temperature for a specified city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city for which to set the temperature"
            },
            "temp": {
                "type": "number",
                "description": "The temperature value to set for the city"
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}

In [4]:
import fastmcp
print(fastmcp.__version__)

2.10.4


In [16]:
# Create the weather_server.py file without emoji
with open('weather_server.py', 'w', encoding='utf-8') as f:
    f.write('''import random
from fastmcp import FastMCP

known_weather_data = {
    'berlin': 20.0
}

# Use plain text instead of emoji
mcp = FastMCP("Demo Server")

@mcp.tool
def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)


@mcp.tool
def set_weather(city: str, temp: float) -> str:
    """
    Sets the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

if __name__ == "__main__":
    mcp.run()
''')

print("weather_server.py file created successfully")

# Now run the server in a subprocess and capture the initial output
import subprocess
import time
import sys

# Start the server process
server_process = subprocess.Popen(
    ["python", "weather_server.py"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

# Give it a moment to start
time.sleep(1)

# Read the initial output from stderr (where FastMCP prints its startup message)
initial_output = server_process.stderr.readline().strip()
print("Server output:", initial_output)

# Terminate the server process after capturing the output
server_process.terminate()
server_process.wait()

# Extract the transport type from the output
import re
transport_match = re.search(r"transport '(\w+)'", initial_output)
if transport_match:
    transport_type = transport_match.group(1)
    print(f"\nThe transport type is: '{transport_type}'")
    print(f"\nAnswer for Q4: {transport_type}")
else:
    print("\nCouldn't extract transport type from output. The full output was:")
    print(initial_output)

weather_server.py file created successfully
Server output: Traceback (most recent call last):

Couldn't extract transport type from output. The full output was:
Traceback (most recent call last):


In [13]:
import asyncio
from fastmcp import Client
import weather_server  # Import our weather server module
# For Jupyter notebook
async def main():
    async with Client(weather_server.mcp) as mcp_client:
        tools = await mcp_client.tools.list()
        print(tools)